In [5]:
import os 
path = '../posts/'
filenames = [name for name in os.listdir(path)]

In [6]:
from pandas import read_csv
df = read_csv('./risk/posts_data.csv')

In [7]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33863 entries, 0 to 33862
Data columns (total 13 columns):
Unnamed: 0      33863 non-null int64
ID              33863 non-null object
author          33863 non-null object
date            33863 non-null object
img             33863 non-null bool
rating          33863 non-null int64
tag_activity    30506 non-null object
tag_category    20381 non-null object
tag_group       6556 non-null object
tag_place       4463 non-null object
tag_xtype       1638 non-null object
title           33863 non-null object
video           33863 non-null bool
dtypes: bool(2), int64(2), object(9)
memory usage: 2.9+ MB


,Unnamed: 0,ID,author,date,img,rating,tag_activity,tag_category,tag_group,tag_place,tag_xtype,title,video
0,0,blog-215386.txt,/people/kate10,10.10.2018,True,25,Альпинизм,Новости,NaN,NaN,NaN,"Жетон ""Спасение в Горах"" снова в Москве",False
1,1,blog-215345.txt,/people/redakciyasajta,03.10.2018,True,77,Воздух,Взаимопомощь,NaN,NaN,NaN,Леониду Мартынову нужна помощь!,False
2,2,blog-215348.txt,/people/redakciyasajta,03.10.2018,True,73,Альпинизм,Новости,NaN,NaN,NaN,Виталий и Антон возвращаются!,False
3,3,blog-215335.txt,/people/robinsya,02.10.2018,True,88,Альпинизм,Помним,NaN,NaN,NaN,Марафон памяти – марафон дружбы,False
4,4,blog-215186.txt,/people/sergeyglaznov,05.09.2018,True,253,Альпинизм,Новости,NaN,NaN,NaN,Хочу заступиться за Пакистанцев!,False


In [8]:
# Проверяем, что нет лишних или недостающих файлов
assert(len(set(filenames).intersection(set(df['ID']))) == len(filenames))

## Токенизация корпуса


### Сегментация

In [ ]:
import nltk

tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
coords = dict()
for name in filenames:
    with open(path + name, 'r') as f:
        coords[name] = '\n'.join([str(span) for span in tokenizer.span_tokenize(f.read())])

In [110]:
from pandas import DataFrame

df_markup = DataFrame.from_dict(coords, orient='index',columns=['coords_sent'])
df_sent.to_csv('./sentences.csv')

In [111]:
sentence_count = 0
for coords in df_markup['coords_sent']:
    sentence_count += len(coords.split('\n'))

In [113]:
# Количество предложений в корпусе:
sentence_count

1016341

### Токенизация

In [10]:
import nltk
import string

def tokenize(text):
    tokens = nltk.word_tokenize(text, 'russian')
    tokens = [word.lower() for word in tokens if ( word not in string.punctuation + '«»–' )]
    return tokens

In [157]:
words = set()
words_count = 0
for name in filenames:
    tokens = tokenize(open(path + name, 'r').read())
    words.update(tokens)
    words_count += len(tokens)

In [158]:
print('Всего слов: ', words_count)
print('Различных слов: ', len(words))

Всего слов:  13348440
Различных слов:  544005


## POS-tagging

In [3]:
import json
import pymorphy2
import nltk

In [76]:
def parse_tag(tag):
    tags_dict = dict()
    tags_dict['POS'] = tag.POS
    tags_dict['animacy'] = tag.animacy
    tags_dict['aspect'] = tag.aspect        # вид: совершенный или несовершенный
    tags_dict['case'] = tag.case          # падеж
    tags_dict['gender'] = tag.gender        # род (мужской, женский, средний)
    tags_dict['involvement'] = tag.involvement   # включенность говорящего в действие
    tags_dict['mood'] = tag.mood          # наклонение (повелительное, изъявительное)
    tags_dict['number'] = tag.number        # число (единственное, множественное)
    tags_dict['person'] = tag.person        # лицо (1, 2, 3)
    tags_dict['tense'] = tag.tense         # время (настоящее, прошедшее, будущее)
    tags_dict['transitivity'] = tag.transitivity  # переходность (переходный, непереходный)
    tags_dict['voice'] = tag.voice         # залог (действительный, страдательный)
    return {k:v for k,v in tags_dict.items() if v is not None}

In [ ]:
morph = pymorphy2.MorphAnalyzer()
tags = dict()
for name in filenames:
    with open(path + name, 'r') as f:
        tokens = [x for x in tokenize(f.read()) if x not in list(string.punctuation + '«»–')]
        tags = {word:tag for word, tag in zip(tokens, [parse_tag(morph.tag(word)[0]) for word in tokens]) if len(tag) > 0}
        with open('../Markup/' + name.replace('txt', 'json'), 'w') as f:
            f.write(json.dumps(tags))

In [ ]:
tag = pymorphy2.tagset.OpencorporaTag
morph_path = '../Markup/'
CASES = tag.CASES
case_count = {k:v for k, v in zip(list(CASES),np.zeros(len(CASES)))}
POS = tag.PARTS_OF_SPEECH
pos_count =  {k:v for k, v in zip(list(POS),np.zeros(len(POS)))}
TENSE = list(tag.TENSES)
tense_count = {k:v for k, v in zip(TENSE, np.zeros(len(TENSE)))}
voice_count = {k:v for k,v in zip(tag.VOICES, np.zeros(len(tag.VOICES)))}
total = 0
for name in os.listdir(morph_path)[1:]:
    with open(morph_path + name, 'r') as f:
        d = json.load(f)
        for key, value in d.items():
            if 'case' in value:
                case_count[value['case']] += 1
            if 'POS' in value:
                pos_count[value['POS']] += 1
            if 'tense' in value:
                tense_count[value['tense']] += 1
            if 'voice' in value:
                voice_count[value['voice']] += 1
            total += 1


In [122]:
case_count_ = sorted(case_count.items(), key=lambda x: x[1])
[(k, v/total) for k, v in case_count_]

[('loc1', 0.0),
 ('acc2', 0.0),
 ('gen1', 0.0),
 ('voct', 8.72685665895525e-05),
 ('gen2', 0.0002976289076588133),
 ('loc2', 0.00224878167424745),
 ('datv', 0.039586260803188646),
 ('ablt', 0.05108416379878993),
 ('loct', 0.05676981865107037),
 ('accs', 0.06698051028947576),
 ('gent', 0.1999395584372139),
 ('nomn', 0.21909110326591835)]

In [123]:
[(k, v/total) for k, v in sorted(pos_count.items(), key=lambda x: x[1])]

[('INTJ', 0.0024081006777589326),
 ('PRED', 0.0040152967791165246),
 ('COMP', 0.0056000023702573645),
 ('NUMR', 0.006249533692265716),
 ('PRTS', 0.006930713337034167),
 ('GRND', 0.00860363021076167),
 ('ADJS', 0.011129435714445399),
 ('PRTF', 0.014478905652174967),
 ('PRCL', 0.01610522542090249),
 ('NPRO', 0.02656465940883088),
 ('CONJ', 0.028705567438266244),
 ('INFN', 0.0392783966932755),
 ('PREP', 0.03963811018302966),
 ('ADVB', 0.06500040267440448),
 ('VERB', 0.1365000241065948),
 ('ADJF', 0.16401265178737606),
 ('NOUN', 0.4247793438535052)]

In [124]:
tenses = 0
for item in tense_count.values():
    tenses += int(item)

In [125]:
[(k, v/tenses) for k, v in sorted(tense_count.items(), key=lambda x: x[1])]

[('futr', 0.08180737680329876),
 ('pres', 0.3492923010067827),
 ('past', 0.5689003221899186)]